In [ ]:
!pip install window-slider

You should consider upgrading via the '/home/abilasha/anaconda3/bin/python -m pip install --upgrade pip' command.


In [ ]:
import pathlib
import numpy as np
import pandas as pd
import json
import math
import scipy.io
from numpy import linalg as LA
import matplotlib.pyplot as plt
import os
from os.path import join, getsize
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc, roc_auc_score, precision_recall_curve, average_precision_score, cohen_kappa_score,confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from window_slider import Slider
import os
import pathlib
import sys
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import pathlib
import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
import json
import scipy.io
from numpy import linalg as LA
import matplotlib.pyplot as plt
import os
from os.path import join, getsize
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc, roc_auc_score, precision_recall_curve, average_precision_score, cohen_kappa_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import time
from sklearn.neighbors import NearestNeighbors

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
bucket_size = 25
length_data=dict()


In [ ]:


def ReadNABDataset(_file_name, _normalize=True):
    with open('/dataset/labels/combined_windows.json') as data_file:
        json_label = json.load(data_file)
    abnormal = pd.read_csv(_file_name, header=0, index_col=0)
    abnormal['label'] = -1
    # abnormal['label'] = 1
    dir_name="/dataset/data_warp"
    list_windows = json_label.get(os.path.relpath(_file_name,dir_name))
    for window in list_windows:
        start = window[0]
        end = window[1]
        abnormal.loc[start:end, 'label'] = 1
        # abnormal.loc[start:end, 'label'] = -1

    abnormal_data = abnormal['value'].as_matrix()
    
    # abnormal_preprocessing_data = np.reshape(abnormal_preprocessing_data, (abnormal_preprocessing_data.shape[0], 1))
    abnormal_label = abnormal['label'].as_matrix()

    print("org len",len(abnormal_data))
    print("anomaly",(len(np.where(abnormal_label==1)[0])))
    # abnormal_data = np.expand_dims(abnormal_data, axis=1)
    # abnormal_label = np.expand_dims(abnormal_label, axis=1)

    
    
    # abnorm_data=np.zeros((len(abnormal_data),bucket_size)) 
    abnorm_data=np.empty((0,bucket_size),float)
    overlap_count =24
    # overlap_count = 0
    slider = Slider(bucket_size,overlap_count)
    slider.fit(abnormal_data)
          
    while True:
        window_data1 = slider.slide()
        
        # do your stuff
        if not (len(window_data1)==bucket_size):
          # print("data",np.shape(window_data1))
          window_data=np.pad(window_data1,[0,(bucket_size-len(window_data1))],mode='constant')
          abnorm_data= np.append(abnorm_data,[window_data], axis=0)
          # print(np.shape(abnorm_data))
          break
        else:
          # abnorm_data[i]=window_data1
          abnorm_data=np.append(abnorm_data,[window_data1], axis=0)
        
        

    # abnormal_data = np.expand_dims(abnorm_data, axis=1)


    # abnorm_label=np.zeros((len(abnormal_label),bucket_size)) 

    #-------------------------------------------------------------------------------------
    abnorm_label=np.empty((0,bucket_size),float)
    slider = Slider(bucket_size,overlap_count)
    slider.fit(abnormal_label)
           
    while True:
        window_label1 = slider.slide()
        
        # do your stuff
        if not (len(window_label1)==bucket_size):
          # print(np.shape(window_label1))
          window_label=np.pad(window_label1,[0,(bucket_size-len(window_label1))],mode='constant',constant_values=-1)
          abnorm_label=np.append(abnorm_label,[window_label], axis=0)
          # print(np.shape(abnorm_label[i]))
          break
        else:
          # abnorm_label[i]=window_label1
          abnorm_label=np.append(abnorm_label,[window_label1], axis=0)

    label=-1*np.ones(len(abnorm_label)) 
    # label=np.ones(len(abnorm_label))   
    for i in range(len(abnorm_label)):
      for j in abnorm_label[i]:
        if j==1:
          label[i]=1
          # label[i]=-1
          break
    print("anom rep",len(np.where(label==1)[0]))
    #-------------------------------------------------------------------


    # abnormal_label = np.expand_dims(abnorm_label, axis=1)

    # print((abnorm_data))

    # if _normalize==True:
    #     scaler = MinMaxScaler(feature_range=(0, 1))
    #     abnormal_data = scaler.fit_transform(abnormal_data)

    # Normal = 1, Abnormal = -1
    # return abnorm_data, label
    return abnorm_data,len(abnormal_data),abnormal_label




In [ ]:


def ReadDataset_withlab(_file_name, _normalize=True):
 
    abnormal = pd.read_csv(_file_name)
    abnormal=abnormal.values
    


    abnormal_data = abnormal[:,0]
    
    # abnormal_preprocessing_data = np.reshape(abnormal_preprocessing_data, (abnormal_preprocessing_data.shape[0], 1))
    abnormal_label = abnormal[:,1]

    print("org len",len(abnormal_data))
    print("anomaly",(len(np.where(abnormal_label==1)[0])))

    abnorm_data=np.empty((0,bucket_size),float)
    overlap_count =24
    # overlap_count = 0
    slider = Slider(bucket_size,overlap_count)
    slider.fit(abnormal_data)
          
    while True:
        window_data1 = slider.slide()
        
        # do your stuff
        if not (len(window_data1)==bucket_size):
          # print("data",np.shape(window_data1))
          window_data=np.pad(window_data1,[0,(bucket_size-len(window_data1))],mode='constant')
          abnorm_data= np.append(abnorm_data,[window_data], axis=0)
          # print(np.shape(abnorm_data))
          break
        else:
          # abnorm_data[i]=window_data1
          abnorm_data=np.append(abnorm_data,[window_data1], axis=0)
        
        

    # abnormal_data = np.expand_dims(abnorm_data, axis=1)


    # abnorm_label=np.zeros((len(abnormal_label),bucket_size)) 

    #-------------------------------------------------------------------------------------
    abnorm_label=np.empty((0,bucket_size),float)
    slider = Slider(bucket_size,overlap_count)
    slider.fit(abnormal_label)
           
    while True:
        window_label1 = slider.slide()
        
        # do your stuff
        if not (len(window_label1)==bucket_size):
          # print(np.shape(window_label1))
          window_label=np.pad(window_label1,[0,(bucket_size-len(window_label1))],mode='constant',constant_values=-1)
          abnorm_label=np.append(abnorm_label,[window_label], axis=0)
          # print(np.shape(abnorm_label[i]))
          break
        else:
          # abnorm_label[i]=window_label1
          abnorm_label=np.append(abnorm_label,[window_label1], axis=0)

    label=-1*np.ones(len(abnorm_label)) 
    # label=np.ones(len(abnorm_label))   
    for i in range(len(abnorm_label)):
      for j in abnorm_label[i]:
        if j==1:
          label[i]=1
          # label[i]=-1
          break
    print("anom rep",len(np.where(label==1)[0]))
    #-------------------------------------------------------------------


    # abnormal_label = np.expand_dims(abnorm_label, axis=1)

    # print((abnorm_data))

    # if _normalize==True:
    #     scaler = MinMaxScaler(feature_range=(0, 1))
    #     abnormal_data = scaler.fit_transform(abnormal_data)

    # Normal = 1, Abnormal = -1
    # return abnorm_data, label
    return abnorm_data,len(abnormal_data),abnormal_label




In [ ]:

if __name__ == '__main__':
    
              for root, dirs, files in os.walk('/dataset/data_warp'):
#                     dirs=['artificial']
                    dirs=['realKnownCause','realAWSCloudwatch','realAdExchange','realTraffic','realTweets']
                    for dir in dirs:
                        
                        s_precision = []
                        s_recall = []
                        s_f1 = []
                        s_roc_auc = []
                        s_pr_auc = []
                        s_cks = []
                        
                        print("root=",root)
                        print("dir=",dir)
                        
                        for root1,dir1,files in os.walk(os.path.join(root,dir)):
                             print(files)
#                              files=['artificial_point.csv']
                             for file in files:   
                                file_name = os.path.join(root,dir,file)
                                print(file_name)
                                
                                abnormal_data,length,abnormal_label = ReadNABDataset(file_name)
#                                 abnormal_data,length,abnormal_label = ReadDataset_withlab(file_name)
                                length_data[file.split('.csv')[-2]]=length
                                print(np.shape(abnormal_data))
                                print(np.shape(abnormal_label))
                                file_name1=os.path.join(os.path.join(root,dir),"out_warp",str(os.path.splitext(file)[0])+"_warp.csv")
                                np.savetxt(file_name1, abnormal_data, delimiter=',',fmt='%10.5f')
#                                 file_name2=os.path.join(os.path.join(root,dir),"out_warp",str(os.path.splitext(file)[0])+"_warp_label.csv")
#                                 # file_name2=os.path.join(os.path.join(root,dir),"out_warp",str(os.path.splitext(file)[0])+"_warp_revlabel.csv")
#                                 np.savetxt(file_name2, abnormal_label, delimiter=',',fmt='%d')
                             break    
                    break

    

In [ ]:


def ReadDataset_rep(_file_name,rep,root,dir, _normalize=True):


    df = pd.read_csv(_file_name,header=None)
    abnormal_data = df.values

    # Normal = 1, Abnormal = -1
    
    return abnormal_data




In [ ]:
def apprecons(y_pred_,length,width):
  y_pred=np.zeros(len(abnormal_data)*len(abnormal_data[0]))
  count=np.zeros(len(abnormal_data)*len(abnormal_data[0]))
  j=0

  for i in range(len(y_pred_)):
    temp=np.copy(count)
    count[j:(width+j)]+=y_pred_[i]
    j+=1

    
  y_pred=np.sign(count)
  # y_pred=np.where(y_pred==0, -1, y_pred)
  y_pred[y_pred ==0] = -1

  # y_pred=np.where(y_pred==0, 1, y_pred)
  return y_pred[0:length]

In [ ]:
def apprecons1(score_,length,width):
  score=np.zeros(len(score_)*len(score_[0]))
  count=np.ones(len(score_)*len(score_[0]))
  j=0
  for i in range(len(score_)):
    temp=np.copy(score)
    score[j:(len(score_[i])+j)]+=score_[i]
    
#     index=np.argwhere(score!=temp)
    j+=1
#     count[index]+=1
#   # print(len(np.argwhere(count[0:length]==0)))
    
  score/=count
#   weights = [0.1,0.15,0.2,0.1,0.2,0.15,0.1]
#   weights = [0.35,0.3,0.35] 
  weights = [0.2,0.25,0.1,0.25,0.2]

#   score = np.convolve(score,np.array(weights)[::-1],'same')
  return score[0:length]


renaming files

In [ ]:
if __name__ == '__main__':
              
              for root, dirs, files in os.walk('/wartem/representations/'):

                    for dir in dirs:
                        print("root=",root)
                        print("dir=",dir)
                        warp="interpolation"
                        for root1,dir1,files in os.walk(os.path.join(root,dir,warp)):
                            print(files)
                            for file in files:
                                file_name = str(dir+'_rep.csv')
                                print(file_name)
                                os.rename(root+dir+'/'+warp+'/'+file,root+dir+'/'+warp+'/'+file_name)
                                

# **NN Method warp**

In [ ]:
def RunModel(_file_name,root,dir,lenth):
    rep=1
    abnormal_data = ReadDataset_rep(_file_name,rep,root,dir) 
    y_pred_arr1= np.empty((0,lenth), int)
    y_predtemp_1=np.zeros((len(abnormal_data),bucket_size))
    y_pred_1=-1*np.ones(lenth,int)
    score_1=np.zeros(lenth)
    y_pred_arr2= np.empty((0,lenth), int)
    y_predtemp_2=np.zeros((len(abnormal_data),bucket_size))
    score_2=np.zeros(lenth)
    y_pred_2=-1*np.ones(lenth,int)
    y_pred_arr3= np.empty((0,lenth), int)
    y_predtemp_3=np.zeros((len(abnormal_data),bucket_size))
    y_pred_3=-1*np.ones(lenth,int)
    score_3=np.zeros(lenth)
    
    d_nn1=[np.array(0) for i in range(len(abnormal_data))]
    d_nn2=[np.array(0) for i in range(len(abnormal_data))]
    d_nn3=[np.array(0) for i in range(len(abnormal_data))]
    # d_nn3_arg=[np.array(0) for i in range(len(abnormal_data))]
    # d_nn3_max=[np.array(0) for i in range(len(abnormal_data))]
    # d_nn3_arg_max=[np.array(0) for i in range(len(abnormal_data))]
    w1=[]
    w2=[]
    w3=[]
    k=0

    mid=math.ceil(bucket_size/2)
    
#     clf = lof.LocalOutlierFactor(n_neighbors=1, n_jobs=2,contamination=0.1)
#     y_temp = clf._fit_predict(abnormal_data)
#     score=clf.negative_outlier_factor_
#     dist=-1*score
#     d_nn1=dist 
#     clf = lof.LocalOutlierFactor(n_neighbors=3, n_jobs=2,contamination=0.1)
#     y_temp = clf._fit_predict(abnormal_data)
#     score=clf.negative_outlier_factor_
#     dist=-1*score 
#     d_nn2=dist
#     clf = lof.LocalOutlierFactor(n_neighbors=5, n_jobs=2,contamination=0.1)
#     y_temp = clf._fit_predict(abnormal_data)
#     score=clf.negative_outlier_factor_
#     dist=-1*score
#     d_nn3=dist

    for r,t in zip(abnormal_data,range(len(abnormal_data))):
            dist=[]
            neigh = NearestNeighbors(n_neighbors=5)
            neigh.fit(abnormal_data)

#......................score to mid....................................   
            y_predtemp_1[k][mid-2:mid+2]=(neigh.kneighbors([r])[0])[0][1]          
            y_predtemp_2[k][mid-2:mid+2]=(neigh.kneighbors([r])[0])[0][2]
            y_predtemp_3[k][mid-2:mid+2]=(neigh.kneighbors([r])[0])[0][4]
#.....................................................................
            # d_nn3_arg[k]=dist_arg[1]
            # d_nn3_max[k]=dist[-1]
            # d_nn3_arg_max[k]=dist_arg[-1]
            # # d_nn3[k]=dist[9]

            k+=1
    
    score_1=apprecons1(y_predtemp_1,lenth,bucket_size)
    score_2=apprecons1(y_predtemp_2,lenth,bucket_size)
    score_3=apprecons1(y_predtemp_3,lenth,bucket_size)
    
    return  score_1,score_2,score_3



if __name__ == '__main__':
              
              for root, dirs, files in os.walk('/dataset/data_warp/'):
                    dirs=['realKnownCause','realAWSCloudwatch','realAdExchange','realTraffic','realTweets']
#                     dirs=['artificial']
                    for dir in dirs:
                        print("root=",root)
                        print("dir=",dir)
                        warp="copy"
                        for root1,dir1,files in os.walk(os.path.join(root,dir)):
                            
#                              files=['artificial_point_warp_rep.csv']
                            print(files)
                            for file in files:
#                                  if file.endswith("_warp_rep.csv"): 
                                dir_=os.path.join(root,dir,warp)
#                          
                                file_name = os.path.join(dir_,file.split('.csv')[0]+'_warp_rep.csv')
                                print("filename",file_name)
                                print(os.path.join(root,dir,file))
                                score_1,score_2,score_3=RunModel(file_name,root,dir,length_data[file.split('.csv')[0]])
                                abnormal_data,lenth,abnormal_label=ReadNABDataset(os.path.join(root,dir,file))
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, score_1)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, score_1)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, score_2)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                               
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, score_3)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))

                            break
                               
                    break         
                                
                                
                                 
                                
